# Ejemplo de uso de Red Neuronal para reconocer imágenes

En este cuaderno implementaremos una **red neuronal simple con PyTorch** para **reconocer dígitos escritos a mano (0 y 1)** del conjunto **MNIST**.  

El objetivo es comprender paso a paso cómo:
- Se **preparan los datos** (transformaciones, lotes y etiquetas),
- Se **define un modelo neuronal** (capas, función de activación y salida),
- Se **entrena y evalúa** mediante una función de pérdida y optimizador.

Este ejemplo aplica los conceptos teóricos de **regresión logística**, **función sigmoide** y **gradiente ascendente**, extendiéndolos al caso de una **red neuronal multicapa** que aprende a clasificar imágenes.


### Importamos librerías necesarias

In [ ]:
import torch                         # Manejo de tensores y operaciones con GPU
import torch.nn as nn                # Módulo para definir redes neuronales
import torch.optim as optim          # Métodos de optimización (SGD, Adam, etc.)
from torchvision import datasets, transforms  # Conjuntos de datos y transformaciones
from torch.utils.data import DataLoader       # Carga de datos por lotes (batch)
import matplotlib.pyplot as plt      # Para graficar imágenes y resultados

### Inicialización de datos de entrenamiento y prueba

In [ ]:
# Inicialización de los datos (entrenamiento y prueba)

# transforms.ToTensor()
# Convierte imágenes en tensores de PyTorch.
# Escala los píxeles de 0–255 a 0.0–1.0 y cambia el formato a (canales, alto, ancho).
transformacion = transforms.ToTensor()

# datasets.MNIST()
# Carga o descarga el conjunto de datos MNIST (dígitos 0–9).
# Devuelve pares (imagen, etiqueta).
# Argumentos:
#   root="./data"          → Carpeta donde se guardan los datos.
#   train=True/False       → Define si son datos de entrenamiento o prueba.
#   download=True          → Descarga automática si no existen.
#   transform=transformacion → Aplica la conversión a tensor.
datos_entrenamiento = datasets.MNIST(
    root="./data", train=True, download=True, transform=transformacion
)

datos_prueba = datasets.MNIST(
    root="./data", train=False, download=True, transform=transformacion
)

# Cada elemento del dataset es una tupla (imagen_tensor, etiqueta)
# Ejemplo: imagen 28x28 y etiqueta numérica entre 0 y 9.


### Máscara de datos para filtrar solo los dígitos 0 y 1

In [ ]:


# Filtrar el dataset para quedarnos solo con los dígitos 0 y 1


# Creamos una "máscara" booleana para cada conjunto:
#   True  → si la etiqueta es 0 o 1
#   False → si la etiqueta es otro número (2–9)
# Esto nos permite seleccionar solo los ejemplos que pertenecen a las clases deseadas.
mascara_entrenamiento = (datos_entrenamiento.targets == 0) | (datos_entrenamiento.targets == 1)
mascara_prueba = (datos_prueba.targets == 0) | (datos_prueba.targets == 1)

# Usamos las máscaras para filtrar los datos e índices correspondientes:
#   - .data almacena las imágenes (matrices 28x28)
#   - .targets contiene las etiquetas (número representado)
datos_entrenamiento.data = datos_entrenamiento.data[mascara_entrenamiento]
datos_entrenamiento.targets = datos_entrenamiento.targets[mascara_entrenamiento]

datos_prueba.data = datos_prueba.data[mascara_prueba]
datos_prueba.targets = datos_prueba.targets[mascara_prueba]

# Ahora ambos conjuntos contienen solo imágenes de los dígitos 0 y 1,
# lo que convierte el problema en una clasificación binaria.



### Definimos características X y etiquetas Y (sin aplanar)

In [ ]:
# Convertimos los datos a tipo flotante para poder operar con PyTorch

# .data contiene las imágenes (matrices 28x28)
# .float() convierte los valores enteros (0–255) a decimales (0.0–255.0)
X_entrenamiento = datos_entrenamiento.data.float()

# .targets contiene las etiquetas (0 o 1)
# .float() las convierte a tipo flotante, útil para operaciones numéricas posteriores
Y_entrenamiento = datos_entrenamiento.targets.float()

# .reshape((n,1)) cambia la forma del tensor a una columna
# Esto permite que Y tenga dimensión [n,1] en lugar de [n], como exige PyTorch
Y_entrenamiento = Y_entrenamiento.reshape((Y_entrenamiento.shape[0], 1))

print("Ejemplo de etiqueta:", Y_entrenamiento[0])

# Repetimos el mismo proceso para el conjunto de prueba


X_prueba = datos_prueba.data.float()
Y_prueba = datos_prueba.targets.float()
Y_prueba = Y_prueba.reshape((Y_prueba.shape[0], 1))


### Definimos modelo, función de pérdida y optimizador

In [ ]:
# Modelo: red totalmente conectada para clasificar 0 vs 1


modelo = nn.Sequential(
    nn.Flatten(),           # Convierte imagen 1x28x28 → vector 784
    nn.Linear(28*28, 512),  # Capa oculta: 784 → 512
    nn.Sigmoid(),           # Activación (no linealidad)
    nn.Linear(512, 1),      # Capa de salida: 512 → 1 (probabilidad de clase 1)
    nn.Sigmoid()            # Mapea salida a [0,1] para clasificación binaria
)

# Función de pérdida y optimizador

fn_perdida = nn.BCELoss()                     # Binary Cross-Entropy para etiquetas {0,1}
optimizador = torch.optim.SGD(                 # Descenso por gradiente (SGD)
    modelo.parameters(), lr=0.001              # lr: tasa de aprendizaje
)

# Nota: alternativamente se puede usar nn.BCEWithLogitsLoss()
# y quitar la última Sigmoid() para mayor estabilidad numérica.


### Entrenamiento del modelo

In [ ]:
# Bucle de entrenamiento: el modelo aprende ajustando sus pesos


total_iteraciones = 30  # Número de veces que veremos todos los datos (épocas)

for i in range(total_iteraciones):

    # Forward pass: calculamos las predicciones de la red
    y_pred = modelo(X_entrenamiento)

    # Cálculo de la pérdida: mide qué tan lejos están las predicciones de las etiquetas reales
    perdida = fn_perdida(y_pred, Y_entrenamiento)

    # Reinicia los gradientes acumulados del paso anterior
    optimizador.zero_grad()

    # Backpropagation: calcula los gradientes ∂L/∂w para cada peso del modelo
    perdida.backward()

    # Actualiza los pesos según el gradiente y la tasa de aprendizaje
    optimizador.step()

    # Muestra el progreso y el valor actual de la pérdida
    print(f"Iteración {i+1}/{total_iteraciones}, Pérdida: {perdida.item():.4f}")


### Evaluación del modelo en datos de prueba

In [ ]:
# Evaluación del modelo: medimos qué tan bien aprendió


modelo.eval()  # Cambia el modelo a modo de evaluación (desactiva dropout, batchnorm, etc.)

# torch.no_grad() desactiva el cálculo de gradientes para ahorrar memoria y acelerar la inferencia.
with torch.no_grad():
    # Calculamos las salidas (predicciones continuas entre 0 y 1)
    salida = modelo(X_prueba)

    # Convertimos las probabilidades en etiquetas binarias:
    #   > 0.5 → clase 1,  ≤ 0.5 → clase 0
    predicciones = (salida > 0.5).float()

    # Comparamos predicciones con etiquetas verdaderas
    # .sum() cuenta cuántas predicciones son correctas
    # .item() convierte el resultado tensorial a número normal (Python float)
    correctas = (predicciones == Y_prueba).sum().item()

    # .size(0) devuelve el total de muestras evaluadas
    total = Y_prueba.size(0)

# Calculamos el porcentaje de aciertos
print(f"Precisión en prueba: {100 * correctas / total:.2f}%")


### Pruebas individuales del modelo

In [ ]:
# Prueba individual: verificamos la predicción para una sola imagen


# Seleccionamos una imagen del conjunto de prueba
indice = 1
imagen = X_prueba[indice]      # Tensor de la imagen seleccionada
etiqueta = Y_prueba[indice]    # Etiqueta real (0 o 1)


# Visualizamos la imagen en escala de grises

plt.imshow(imagen.reshape(28, 28), cmap='gray')    # Convertimos a 28x28 para mostrar
plt.title(f"Etiqueta real: {int(etiqueta)}")       # Mostramos la clase verdadera
plt.show()
# Evaluamos el modelo con esa imagen
modelo.eval()  # Modo evaluación (sin actualización de pesos)

with torch.no_grad():  # Sin cálculo de gradientes (ahorra memoria)
    entrada = imagen.reshape(1, 28, 28)    # Redimensionamos para simular un lote (batch=1)
    salida = modelo(entrada)               # Forward pass → salida del modelo
    prediccion = (salida > 0.5).float()    # Clasificación: 1 si probabilidad > 0.5, de lo contrario 0

# Mostramos el resultado del modelo
print(f"Predicción del modelo: {int(prediccion.item())}")
